<a href="https://colab.research.google.com/github/jeperniag/BussinessIntelligence/blob/main/Reto_2_%C3%81rboles_de_Decisi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Se procede a cargar las librerias de trabajo

In [ ]:
#Librerias cientificas
import numpy as np
import pandas as pd

#Librerias Arbol
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Librerias Visualizacion
import matplotlib.pyplot as plt
import seaborn as sns

#Conectar al Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Se procede a cargar la base de datos

In [ ]:
nxl='/content/drive/MyDrive/2. CollegeAdmisions_Int_M.xlsx'
XDB=pd.read_excel(nxl, sheet_name=0) #College_Admisions
XDB=XDB.dropna() #Eliminar filas vacías
XDB.head(100) #Mostrar 100 lineas del archivo

#Convertir la varible Gender a numérica
XDB['Gender'] = XDB['Gender'].map({'F': 0, 'M': 1})

#Seleccionar las variables de interés
XD=XDB[['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT']]
yd=XDB[['Admitted']]


,Admitted
6,1
28,1
46,0
61,0
65,1
...,...
17330,1
17332,1
17333,1
17334,1


**Descripción de los Datos**
De acuerdo con la base de datos de estudiantes admitidos en la universidad, esta posee un total de 1528 datos totales, divividos en 225 estudiantes no admitidos y 1303 personas admitidas. Cada uno de los estudiantes estará descrito en función de 7 variables:

* Edu_Parent1: Hace referencia al nivel educativo que posee su primer padre.
* Edu_Parent2: Hace referencia al nivel educativo que posee su segundo padre.
* Gender: Hace referencia al género del estudiante interesado en ingresar a la universidad. (F=0 y M=1)
* White: Variable que indica si el estudiante es idenficiado como blanco. (Si=1 y No=0)
* Asian: Variable que indica si el estudiante es idenficiado como asiatico. (Si=1 y No=0)
* HSGPA: Indica el promedio de calificaciones de los estudiantes en su secundaria.
* SAT/ACT: Puntaje obtenido por los estudiantes en los examénes SAT o ACT.
* Admitted: Variable que indica si el estudiante fue admitido o no. (Si=1 y No=0)

* Se determinan el número de personas admitidas en la Universidad

In [ ]:
# Convertir 'Yes' a 1 y 'No' a 0
XDB["Admitted"] = XDB["Admitted"].map({"Yes": 1, "No": 0})

# Ahora sí, contar los admitidos
num_admitidos = XDB["Admitted"].sum()
print(f"Número de personas admitidas: {num_admitidos}")

Número de personas admitidas: 1303


2. Se procede con la implementación del modelo

In [ ]:
mar=DecisionTreeClassifier(criterion='gini',max_depth=4)
mar.fit(XD,yd) #Ajustar el modelo entre la entrada y la salida

#Se procede con la gráfica
from pydotplus import graph_from_dot_data #Me crea la gráfica
from sklearn.tree import export_graphviz #Me exporta a PNG, JPEG, TIFF.

ve=['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT'] #Nombres para el Árbol
dot_data=export_graphviz(mar,feature_names=ve) #Crear el gráfico
graph=graph_from_dot_data(dot_data)
graph.write_png('2. ArbolDecision_M.png') #Exportar a PNG

True

* Se determina la cantidad de nodos puros para el árbol del modelo

In [ ]:
num_nodos_puros = sum(mar.tree_.impurity == 0)
print(f"Número de nodos puros en el árbol (Gini = 0.0): {num_nodos_puros}")

Número de nodos puros en el árbol (Gini = 0.0): 7


3. Se procede a realizar el pronóstico de los solicitantes que no fueron admitidos

In [ ]:
#Se carga la hoja
XDB0=pd.read_excel(nxl, sheet_name=1)
XDB0.head(100) #Mostrar 100 lineas del archivo

#Se convierte gender a numérico
XDB0['Gender'] = XDB0['Gender'].map({'F': 0, 'M': 1})

#Se crea la base de datos a pronósticar
XD0=XDB0[['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT']]
yd0=XDB0[['Admitted']]

#Mantener a todos los solicitantes
filas=XDB0.index

#Le calculamos la predicción
y_pred=mar.predict(XD0)

#Se crea el informe en Excel
df=pd.DataFrame(XD0)
df['Admitted_Predicted']=y_pred
df.insert(0,'Applicant',filas)

#Guardar Excel
df.to_excel('2. ArbolDecision_M.xlsx', index=False)

4. Evaluar el comportamiento del modelo

In [ ]:
ydp=mar.predict(XD) #Prónostico sobre la base de datos original
cm=confusion_matrix(yd,ydp) #Matriz de confusión
print("La matriz de confusión es:\n", cm)

VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]

Exactitud=(VP+VN)/(VP+VN+FP+FN)
Sensibilidad=VP/(VP+FN)
Especificidad=VN/(VN+FP)
Precisión=VP/(VP+FP)
TasaError=(FP+FN)/(VP+VN+FP+FN)
PredNeg=VN/(VN+FN) #Tasa de F
PredPos=VP/(VP+FP) #Tasa de V

print("La exactitud es:",Exactitud)
print("La sensibilidad es:",Sensibilidad)
print("La especificidad es:",Especificidad)
print("La precisión es:",Precisión)
print("La tasa de error es:",TasaError)
print("La tasa de predicción negativa es:",PredNeg)

La matriz de confusión es:
 [[  20  205]
 [   2 1301]]
La exactitud es: 0.8645287958115183
La sensibilidad es: 0.9984650805832693
La especificidad es: 0.08888888888888889
La precisión es: 0.8638778220451527
La tasa de error es: 0.1354712041884817
La tasa de predicción negativa es: 0.9090909090909091


**Análisis de los Resultados**
La matriz de confusión muestra una alta predominancia por los valores verdaderos positivos (1301) y verdaderos negativos (20), mientras que los falsos positivos (205) y falsos negativos (2) son significativamente menores respecto a los positivos. Esto hizo que el modelo tuviera un buen desempeño en la identificación de estudiantes admitidos, aunque su capacidad para detectar correctamente a los no admitidos es baja.

Asimismo, su exactitud cerca al 90% (86,4%), nos da a entender que es un modelo que se comporta bien a la hora de indentificar a los estudiantes que son admitidos. Además, esto lo corrobora su tasa de error del 13% que es baja, de igual manera, también podemos observar la sensibilidad que nos da a entender que el modelo funciona correctamente a la hora de indentificar a los estudiantes admitidos.

Sin embargo, se tiene una especificidad bastante baja del 8% que nos da a entender que si bien el modelo es bueno para identificar a los admitidos no es tan bueno para identificar a los que no lo fueron.

Finalmente, se puede decir que su comportamieto es correcto aunque haya margen de mejora en los que no fueron admitidos, ya que todos los resultados nos dieron un porcentaje favorable excepto el de especificidad.
